In [1]:
from model.BERTNoiseDefend import BertForSequenceClassification
from model.RoBERTaNoiseDefend import RobertaForSequenceClassification
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from textattack.models.wrappers import (
    ModelWrapper,
    PyTorchModelWrapper,
    SklearnModelWrapper,
    HuggingFaceModelWrapper,
)


config = AutoConfig.from_pretrained("model/weights/bert-base-uncased-imdb")
tokenizer_tmd = AutoTokenizer.from_pretrained(
    "model/weights/bert-base-uncased-imdb", use_fast=True
)

model = BertForSequenceClassification(config)
state = AutoModelForSequenceClassification.from_pretrained(
    "model/weights/bert-base-uncased-imdb"
)
model.load_state_dict(state.state_dict())
model.to("cpu")
model.eval()

tokens = tokenizer_tmd(["This film is terrible. You don't really need to read this review further. If you are planning on watching it, suffice to say - don't (unless you are studying how not to make a good movie).<br /><br />The acting is horrendous... serious amateur hour. Throughout the movie I thought that it was interesting that they found someone who speaks and looks like Michael Madsen, only to find out that it is actually him! A new low even for him!!<br /><br />The plot is terrible. People who claim that it is original or good have probably never seen a decent movie before. Even by the standard of Hollywood action flicks, this is a terrible movie.<br /><br />Don't watch it!!! Go for a jog instead - at least you won't feel like killing yourself.","This film is terrible. You don't really need to read this review further. If you are planning on watching it, suffice to say - don't (unless you are studying how not to make a good movie).<br /><br />The acting is horrendous... serious amateur hour. Throughout the movie I thought that it was interesting that they found someone who speaks and looks like Michael Madsen, only to find out that it is actually him! A new low even for him!!<br /><br />The plot is terrible. People who claim that it is original or good have probably never seen a decent movie before. Even by the standard of Hollywood action flicks, this is a terrible movie.<br /><br />Don't watch it!!! Go for a jog instead - at least you won't feel like killing yourself."],add_special_tokens=True,
            padding="max_length",
            max_length=256,
            truncation=True,
            return_tensors="pt")

output = model(tokens["input_ids"],tokens["token_type_ids"],tokens["attention_mask"])

/opt/conda/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-22 07:58:16.763062: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


OSError: Can't load the configuration of '/home/ubuntu/RobustExperiment/model/weights/VinAI_weights/bert-base-uncased-imdb'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/home/ubuntu/RobustExperiment/model/weights/VinAI_weights/bert-base-uncased-imdb' is the correct path to a directory containing a config.json file

In [3]:
for layers in model.bert.encoder.layer:
    print(layers.attention_output.shape)

torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])


In [ ]:
import torch
from tqdm import tqdm
from utils.dataloader import load_train_test_imdb_data
import numpy as np

num_repetitions = 3
batch=500
y_pred_BERT = []
train_data, test_data = load_train_test_imdb_data(
        "data/aclImdb"
    )
test_labels = np.array(test_data["label"])
bert_input = list(test_data["text"])
device = "cuda"
BERT = HuggingFaceModelWrapper(model,tokenizer_tmd)
saved_features = []
for i in tqdm(range(0,len(bert_input)//batch)):
    BERT(bert_input[i*batch:(i+1)*batch])
    saved_features.append(model.bert.encoder.layer[0].attention_output)